# Composer classification using a CNN and a LSTM in tensorflow

In [69]:
import pandas as pd
from mido import MidiFile
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import silhouette_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [51]:
# loading the data into four different dataframes for the composers midi files
def midi_to_dataframe(midi_path):
    try:
        midi = MidiFile(midi_path)
        data = []
        for i, track in enumerate(midi.tracks):
            for msg in track:
                if not msg.is_meta:
                    data.append([i, msg.type, msg.time, msg.dict()])
        df = pd.DataFrame(data, columns=['track', 'type', 'time', 'msg_dict'])
        return df
    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
        return pd.DataFrame()

# process the data frame
def preprocess_dataframe(df):
    msg_df = df['msg_dict'].apply(pd.Series)
    msg_df = msg_df.add_prefix('msg_')
    df = pd.concat([df.drop(columns=['msg_dict']), msg_df], axis=1)
    return df

# load and pre-process da data 
def load_and_preprocess_data(main_pathway, composers):
    composer_dataframes = {}
    for composer in composers:
        folder_path = os.path.join(main_pathway, composer)
        if os.path.exists(folder_path):
            all_midi_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.mid')]
            composer_dfs = [midi_to_dataframe(midi_file) for midi_file in all_midi_files]
            composer_dfs = [df for df in composer_dfs if not df.empty]
            if composer_dfs:
                combined_df = pd.concat(composer_dfs, ignore_index=True)
                combined_df = preprocess_dataframe(combined_df)
                composer_dataframes[composer] = combined_df
    return composer_dataframes


In [52]:
# my file path 
main_pathway = 'C:/Git_hub_repos/aai-511_group1/midiclassics'
composers = ['bach', 'beethoven', 'chopin', 'mozart']

# load da data 
composer_dataframes = load_and_preprocess_data(main_pathway, composers)

# getting da dfs
bach_df = composer_dataframes.get('bach', pd.DataFrame())
beethoven_df = composer_dataframes.get('beethoven', pd.DataFrame())
chopin_df = composer_dataframes.get('chopin', pd.DataFrame())
mozart_df = composer_dataframes.get('mozart', pd.DataFrame())

# print lens of dfs 
len_bach = len(bach_df)
len_beethoven = len(beethoven_df)
len_chopin = len(chopin_df)
len_mozart = len(mozart_df)

print(f"Length of Bach dataframe: {len_bach}")
print(f"Length of Beethoven dataframe: {len_beethoven}")
print(f"Length of Chopin dataframe: {len_chopin}")
print(f"Length of Mozart dataframe: {len_mozart}")

# df head method 
bach_df.head()

Error processing C:/Git_hub_repos/aai-511_group1/midiclassics\beethoven\Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
Length of Bach dataframe: 688603
Length of Beethoven dataframe: 1543439
Length of Chopin dataframe: 730587
Length of Mozart dataframe: 746649


,track,type,time,msg_type,msg_time,msg_note,msg_velocity,msg_channel,msg_program,msg_control,msg_value,msg_pitch
0,1,note_on,0,note_on,0,53.0,30.0,0,NaN,NaN,NaN,NaN
1,1,note_off,48,note_off,48,53.0,30.0,0,NaN,NaN,NaN,NaN
2,1,note_on,0,note_on,0,57.0,30.0,0,NaN,NaN,NaN,NaN
3,1,note_off,48,note_off,48,57.0,30.0,0,NaN,NaN,NaN,NaN
4,1,note_on,0,note_on,0,60.0,30.0,0,NaN,NaN,NaN,NaN


In [53]:
# checker function
def get_unique_values(df, column_name):
    if column_name in df.columns:
        return df[column_name].unique()
    else:
        return []

# function to get all unique values for every df 
def get_unique_values_for_df(df):
    unique_values = {}
    for col in df.columns:
        unique_values[col] = get_unique_values(df, col)
    return unique_values

unique_values_bach = get_unique_values_for_df(bach_df)

# printin the unique values
for key, value in unique_values_bach.items():
    print(f"Unique values for column {key}: {value}")


Unique values for column track: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18  0 19]
Unique values for column type: ['note_on' 'note_off' 'program_change' 'control_change' 'pitchwheel']
Unique values for column time: [   0   48  768 ... 1871 2234 1207]
Unique values for column msg_type: ['note_on' 'note_off' 'program_change' 'control_change' 'pitchwheel']
Unique values for column msg_time: [   0   48  768 ... 1871 2234 1207]
Unique values for column msg_note: [ 53.  57.  60.  65.  69.  55.  62.  67.  70.  52.  58.  59.  64.  50.
  63.  66.  61.  56.  41.  36.  38.  43.  48.  46.  45.  47.  49.  72.
  74.  77.  71.  76.  68.  79.  81.  75.  82.  73.  78.  84.  80.  83.
  nan  51.  44.  39.  54.  40.  42.  33.  26.  93.  95.  97. 102. 100.
  98.  92.  90.  88.  31.  37.  35.  28.  30.  34.  29.  24.  86.  32.
  27.  21.  25.  23.  18.  19.  20.  16.  17.  14.  12.  91.  89.  85.
  87.  94.  96.]
Unique values for column msg_velocity: [ 30.  32.  50.  40.  55.  52.  57.  56.  60.

In [60]:
# finds the max time index in the df's

def find_max_time(df):
    return df['time'].max()


#  function is to process and then group the data, before returning the matrix using the last funct to make sure data is processed correctly
def process_group(group, max_time):
    matrix = np.zeros((max_time + 1, 128))  
    for _, row in group.iterrows():
        if pd.notna(row['msg_note']) and pd.notna(row['msg_velocity']):
            time = int(row['time'])
            note = int(row['msg_note'])
            velocity = row['msg_velocity']
            if 0 <= note < 128:
                matrix[time, note] = velocity
    return matrix


In [61]:
# now this function will actually prepare the data for the model and return the matrix

# Calculating the maximum time from all DataFrames
max_time = max(find_max_time(bach_df), find_max_time(beethoven_df), find_max_time(chopin_df), find_max_time(mozart_df))

# Now use this max_time in the prepare_data function
def prepare_data(df, composer_name, max_time):
    df_sorted = df.sort_values(by=['track', 'time'])
    grouped = df_sorted.groupby('track')
    track_matrices = [process_group(group, max_time) for _, group in grouped]
    X = np.stack(track_matrices)  # Ensures uniform array shape
    y_labels = np.array([composer_name] * len(track_matrices))
    return X, y_labels

In [63]:
bach_X, bach_y = prepare_data(bach_df, 'Bach', max_time)
beethoven_X, beethoven_y = prepare_data(beethoven_df, 'Beethoven', max_time)
chopin_X, chopin_y = prepare_data(chopin_df, 'Chopin', max_time)
mozart_X, mozart_y = prepare_data(mozart_df, 'Mozart', max_time)

In [64]:
X = np.concatenate([bach_X, beethoven_X, chopin_X, mozart_X])
y = np.concatenate([bach_y, beethoven_y, chopin_y, mozart_y])

## `Model building`

In [ ]:
# building just a very simple starting mdoel

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax') 
])

# complile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# error running the cell above

In [70]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)